In [1]:
#!/usr/bin/env python3
import os
import logging
import sys
from pathlib import Path

logging.basicConfig(level=logging.DEBUG)

from Pegasus.api import *

input_file = File("input.txt")
rc = ReplicaCatalog()\
        .add_replica("local", "input.txt", Path(".").resolve() / "input.txt")\
        .write()

tc = TransformationCatalog()
tools_container = Container(
                    "tools-container", 
                    Container.DOCKER, 
                    image="docker:///ssrujanaa/workflow4:latest"
                )
tc.add_containers(tools_container)
tc.write()

Thumbs_up = Transformation(
            "Thumbs_up",
            site="condorpool",
            pfn="/usr/bin/add_thumbs_up.py",
            is_stageable=False,
            container=tools_container
            )

tc.add_transformations(Thumbs_up).write()

wf = Workflow("add_thumbs")

output_file = File("output.txt")

job_add_thumbs = Job(Thumbs_up)\
                    .add_args(input_file, output_file)\
                    .add_inputs(input_file)\
                    .add_outputs(output_file)

wf.add_jobs(job_add_thumbs)

In [2]:
try:
    wf.plan(submit=True)\
        .wait()\
        .analyze()\
        .statistics()
except PegasusClientError as e:
    print(e.output)

Plan:
[main] WARN  schema.JsonMetaSchema  - Unknown keyword $defs - you should define your own Meta Schema. If the keyword is irrelevant for validation, just use a NonValidationKeyword
[main] WARN  schema.JsonMetaSchema  - Unknown keyword additionalItems - you should define your own Meta Schema. If the keyword is irrelevant for validation, just use a NonValidationKeyword
[main] WARN  schema.JsonMetaSchema  - Unknown keyword examples - you should define your own Meta Schema. If the keyword is irrelevant for validation, just use a NonValidationKeyword
2020.07.16 17:53:13.353 UTC:    
2020.07.16 17:53:13.359 UTC:   ----------------------------------------------------------------------- 
2020.07.16 17:53:13.364 UTC:   File for submitting this DAG to HTCondor           : add_thumbs-0.dag.condor.sub 
2020.07.16 17:53:13.369 UTC:   Log of DAGMan debugging messages                 : add_thumbs-0.dag.dagman.out 
2020.07.16 17:53:13.375 UTC:   Log of HTCondor library output                     :

[##################################################] 100.0% ..Success (Completed: 8, Queued: 0, Running: 0, Failed: 0)


Analyzer:
Your database is compatible with Pegasus version: 5.0.0dev

************************************Summary*************************************

 Submit Directory   : /home/scitech/shared-data/Workflow/Workflow4/scitech/pegasus/add_thumbs/run0015
 Total jobs         :      8 (100.00%)
 # jobs succeeded   :      8 (100.00%)
 # jobs failed      :      0 (0.00%)
 # jobs held        :      0 (0.00%)
 # jobs unsubmitted :      0 (0.00%)

 

Statistics:
Your database is compatible with Pegasus version: 5.0.0dev

#
# Pegasus Workflow Management System - http://pegasus.isi.edu
#
# Workflow summary:
#   Summary of the workflow execution. It shows total
#   tasks/jobs/sub workflows run, how many succeeded/failed etc.
#   In case of hierarchical workflow the calculation shows the
#   statistics across all the sub workflows.It shows the following
#   statistics about tasks, jobs and sub workflows.
#     * Succeeded - total count of succeeded tasks/jobs/sub workflows.
#     * Failed - total 